### 1. Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pyodbc
import pandas as pd
import numpy as np
import xlwings as xw

### 2. Import data to Pandas dataframe (Kidos_Kind_Relatie)

In [2]:
# Import pickles
df_schoolgaanden = pd.read_pickle('4_Data\Pickles\schoolgaanden_observaties.pkl')

In [3]:
# Print the amount of unique clients in df
print(f'The amount of unique \'Clientnummer\'s in the df is: {df_schoolgaanden['Clientnummer'].nunique()}')

The amount of unique 'Clientnummer's in the df is: 14047


In [4]:
# Set to True or False depending on whether you want to import new data
NEWIMPORT = False

if NEWIMPORT:
    # Replace server name if source changes
    server = 'AZ-ODB0\ODBWB'
    database = 'OnderzoeksDB_WB'

    # Create a connection string
    connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database}'

    # Establish connection
    connection = pyodbc.connect(connection_string)

    # Define columns to select
    selected_columns = ['Clientnummer',
                        'Relatienummer',
                        'Relatieband',
                        'Ouder',
                        'Geboortejaarmaand_kind',
                        'Geboortejaarmaand_relatie',
                        # Geboorteland_kind,                    Staat volgens mij niet in docx, gewenst?
                        'Geboorteland_relatie',
                        'Geslacht_relatie'
                        ]
    
    # Define a list of Clientnummers to include
    included_clientnummers = df_schoolgaanden['Clientnummer'].unique()

    # Create SQL query string with specific columns and inclusion criteria
    query = f'SELECT {', '.join(selected_columns)} FROM Kidos_Kind_Relatie WHERE Clientnummer IN ({', '.join(included_clientnummers)})'

    # Execute the query and fetch the data into a Pandas DataFrame
    df_relaties_og = pd.read_sql(query, connection)

    # Close the connection
    connection.close()

    # Save df_relaties_og as a pickle
    df_relaties_og.to_pickle('4_Data\Pickles\\relaties_schoolgaanden_og.pkl')
else:
    # Load the pickle
    df_relaties_og = pd.read_pickle('4_Data\Pickles\\relaties_schoolgaanden_og.pkl')

### 3. Process

In [5]:
print(f'The amount of relations in the df_relaties is: {len(df_relaties_og)}, the amount of unique client ID\'s is: {len(df_relaties_og['Clientnummer'].unique())}')

The amount of relations in the df_relaties is: 51090, the amount of unique client ID's is: 13864


In [6]:
# Store copy of unique records of df_og in df
df_relaties = df_relaties_og.drop_duplicates()

# Set yearmonth's to NaN if it is '00'
df_relaties['Geboortejaarmaand_relatie'] = df_relaties['Geboortejaarmaand_relatie'].replace('00', np.NaN)

In [7]:
print(f'The amount of relations in the df_relaties is: {len(df_relaties)}, the amount of unique client ID\'s is: {len(df_relaties['Clientnummer'].unique())}')

The amount of relations in the df_relaties is: 51090, the amount of unique client ID's is: 13864


#### 3.1 Transform table structure - Siblings

In [8]:
# Create a pivot table with 'Clientnummer' as the index and 'Relatieband' as the columns
clients_frequencies = df_relaties.pivot_table(index='Clientnummer', columns='Relatieband', aggfunc='size', fill_value=0)

# Select only the 'Broer of zus' column
clients_relations_siblings = clients_frequencies['Broer of zus'].reset_index(name='Siblings')

# Show head
print(clients_relations_siblings.shape)
clients_relations_siblings.head()

(13864, 2)


,Clientnummer,Siblings
0,10070491,0
1,10078065,3
2,10078073,1
3,2986905,1
4,2987112,1


#### 3.2 Transform table structure - Parent1

In [9]:
# Remove all rows where Relatieband is not in relevant_relaties
df_relaties_parent1 = df_relaties[df_relaties['Ouder'] == 'Ouder1']

# Pivot from long to wide format
clients_relations_parent1 = (df_relaties_parent1.pivot_table(index='Clientnummer',
                                                    columns='Ouder',
                                                    values=['Geboortejaarmaand_relatie', 'Geboorteland_relatie', 'Geslacht_relatie'],
                                                    aggfunc='first')
                                                    .pipe(lambda d: d.set_axis(d.columns.map(lambda x: f'{x[0]}_ouder1'), axis=1))
                                                    .reset_index())

# Show head
print(clients_relations_parent1.shape)
clients_relations_parent1.head()

(13653, 4)


,Clientnummer,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1
0,10070491,198708,Nederland,mannelijk
1,10078065,196703,Turkije,onbekend
2,10078073,198807,China,vrouwelijk
3,2986905,197805,Nederland,vrouwelijk
4,2987112,198604,Nederland,vrouwelijk


In [10]:
clients_relations_parent1.isna().mean()

Clientnummer                        0.000000
Geboortejaarmaand_relatie_ouder1    0.000879
Geboorteland_relatie_ouder1         0.000146
Geslacht_relatie_ouder1             0.000000
dtype: float64

#### 3.3 Transform table structure - Parent2

In [11]:
# Remove all rows where Relatieband is not in relevant_relaties
df_relaties_parent2 = df_relaties[df_relaties['Ouder'] == 'Ouder2']

# Pivot from long to wide format
clients_relations_parent2 = (df_relaties_parent2.pivot_table(index='Clientnummer',
                                                    columns='Ouder',
                                                    values=['Geboortejaarmaand_relatie', 'Geboorteland_relatie', 'Geslacht_relatie'],
                                                    aggfunc='first')
                                                    .pipe(lambda d: d.set_axis(d.columns.map(lambda x: f'{x[0]}_ouder2'), axis=1))
                                                    .reset_index())

# Show head
print(clients_relations_parent2.shape)
clients_relations_parent2.head()

(13539, 4)


,Clientnummer,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
0,10070491,198503,Nederland,vrouwelijk
1,10078065,198005,Turkije,onbekend
2,10078073,198111,China,mannelijk
3,2986905,198201,Nederland,mannelijk
4,2987112,198709,Nederland,mannelijk


In [12]:
clients_relations_parent2.isna().mean()

Clientnummer                        0.000000
Geboortejaarmaand_relatie_ouder2    0.003841
Geboorteland_relatie_ouder2         0.000000
Geslacht_relatie_ouder2             0.000000
dtype: float64

#### 3.4 Merge siblings and parents

In [13]:
# Merge clients_siblings_frequencies and clients_relations on 'Clientnummer' keeping all rows from clients_siblings_frequencies
clients_relations = (clients_relations_siblings
                     .merge(clients_relations_parent1, on='Clientnummer', how='left')
                     .merge(clients_relations_parent2, on='Clientnummer', how='left'))

# Show head
print(clients_relations.shape)
clients_relations.head()

(13864, 8)


,Clientnummer,Siblings,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
0,10070491,0,198708,Nederland,mannelijk,198503,Nederland,vrouwelijk
1,10078065,3,196703,Turkije,onbekend,198005,Turkije,onbekend
2,10078073,1,198807,China,vrouwelijk,198111,China,mannelijk
3,2986905,1,197805,Nederland,vrouwelijk,198201,Nederland,mannelijk
4,2987112,1,198604,Nederland,vrouwelijk,198709,Nederland,mannelijk


#### 3.5 Align fathers with parent2 and mothers with parent1

In [14]:
# Replace all NaN with 'onbekend'
clients_relations = clients_relations.fillna('onbekend')

# If parent1 is masculine and parent2 is feminine or unknown, switch them
mask = ((clients_relations['Geslacht_relatie_ouder1'] == 'mannelijk') &
        ((clients_relations['Geslacht_relatie_ouder2'] == 'vrouwelijk') | (clients_relations['Geslacht_relatie_ouder2'] == 'onbekend')) |
        ((clients_relations['Geslacht_relatie_ouder2'] == 'vrouwelijk') & (clients_relations['Geslacht_relatie_ouder1'] == 'onbekend')))

# Switch
clients_relations.loc[mask, ['Geboortejaarmaand_relatie_ouder1',
                             'Geboorteland_relatie_ouder1',
                             'Geslacht_relatie_ouder1',
                             'Geboortejaarmaand_relatie_ouder2',
                             'Geboorteland_relatie_ouder2',
                             'Geslacht_relatie_ouder2']] = clients_relations.loc[mask, ['Geboortejaarmaand_relatie_ouder2',
                                                                                        'Geboorteland_relatie_ouder2',
                                                                                        'Geslacht_relatie_ouder2',
                                                                                        'Geboortejaarmaand_relatie_ouder1',
                                                                                        'Geboorteland_relatie_ouder1',
                                                                                        'Geslacht_relatie_ouder1']].values

In [15]:
# Show head
print(clients_relations.shape)
clients_relations.head()

(13864, 8)


,Clientnummer,Siblings,Geboortejaarmaand_relatie_ouder1,Geboorteland_relatie_ouder1,Geslacht_relatie_ouder1,Geboortejaarmaand_relatie_ouder2,Geboorteland_relatie_ouder2,Geslacht_relatie_ouder2
0,10070491,0,198503,Nederland,vrouwelijk,198708,Nederland,mannelijk
1,10078065,3,196703,Turkije,onbekend,198005,Turkije,onbekend
2,10078073,1,198807,China,vrouwelijk,198111,China,mannelijk
3,2986905,1,197805,Nederland,vrouwelijk,198201,Nederland,mannelijk
4,2987112,1,198604,Nederland,vrouwelijk,198709,Nederland,mannelijk


### 4. Transform monthyear dates

In [16]:
# Change the data type of the column to string
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].astype(str)
clients_relations['Geboortejaarmaand_relatie_ouder2'] = clients_relations['Geboortejaarmaand_relatie_ouder2'].astype(str)

# If the length of the string is 5, add 1 as its first character
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].apply(lambda x: '1' + x if len(x) == 5 else x)
clients_relations['Geboortejaarmaand_relatie_ouder2'] = clients_relations['Geboortejaarmaand_relatie_ouder2'].apply(lambda x: '1' + x if len(x) == 5 else x)

# If the second character is a 0, replace it with a 9
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].apply(lambda x: x[:1] + '9' + x[2:] if x[1] == '0' else x)
clients_relations['Geboortejaarmaand_relatie_ouder2'] = clients_relations['Geboortejaarmaand_relatie_ouder2'].apply(lambda x: x[:1] + '9' + x[2:] if x[1] == '0' else x)

# Replace "onbekend" with np.nan
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].replace('onbekend', np.nan)
clients_relations['Geboortejaarmaand_relatie_ouder2'] = clients_relations['Geboortejaarmaand_relatie_ouder2'].replace('onbekend', np.nan)

# Replace other individual input errors
clients_relations['Geboortejaarmaand_relatie_ouder1'] = clients_relations['Geboortejaarmaand_relatie_ouder1'].replace('291001', np.nan)

# Change month dtypes
clients_relations['Geboortejaarmaand_relatie_ouder1'] = pd.to_datetime(clients_relations['Geboortejaarmaand_relatie_ouder1'], format='%Y%m').dt.to_period('M').dt.strftime('%Y-%m')
clients_relations['Geboortejaarmaand_relatie_ouder2'] = pd.to_datetime(clients_relations['Geboortejaarmaand_relatie_ouder2'], format='%Y%m').dt.to_period('M').dt.strftime('%Y-%m')

In [17]:
clients_relations.isna().mean()

Clientnummer                        0.000000
Siblings                            0.000000
Geboortejaarmaand_relatie_ouder1    0.012262
Geboorteland_relatie_ouder1         0.000000
Geslacht_relatie_ouder1             0.000000
Geboortejaarmaand_relatie_ouder2    0.031088
Geboorteland_relatie_ouder2         0.000000
Geslacht_relatie_ouder2             0.000000
dtype: float64

### 5. Save as .pkl

In [18]:
# Save df_observaties as .pkl file
clients_relations.to_pickle('4_Data/Pickles/schoolgaanden_relaties.pkl')